<a href="https://colab.research.google.com/github/Indresh0007/PySpark-Indresh/blob/main/Product_Analytics_%26_Workflow_Automation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://downloads.lightbend.com/scala/2.12.10/scala-2.12.10.deb
!dpkg -i scala-2.12.10.deb
!wget -q https://archive.apache.org/dist/spark/spark-3.3.0/spark-3.3.0-bin-hadoop3.tgz
!tar xf spark-3.3.0-bin-hadoop3.tgz
!pip install findspark


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,891 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,161 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-upda

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.0-bin-hadoop3"


In [3]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("ClassicModels Analytics") \
    .getOrCreate()

print("Spark Version:", spark.version)


Spark Version: 3.3.0


In [7]:
from google.colab import files
uploaded = files.upload()


Saving customers.csv to customers.csv
Saving employees.csv to employees.csv
Saving offices.csv to offices.csv
Saving orderdetails.csv to orderdetails.csv
Saving orders.csv to orders.csv
Saving payments.csv to payments.csv
Saving productlines.csv to productlines.csv
Saving products.csv to products.csv


In [8]:
products = spark.read.csv("products.csv", header=True, inferSchema=True)
customers = spark.read.csv("customers.csv", header=True, inferSchema=True)
orders = spark.read.csv("orders.csv", header=True, inferSchema=True)
orderdetails = spark.read.csv("orderdetails.csv", header=True, inferSchema=True)
payments = spark.read.csv("payments.csv", header=True, inferSchema=True)
productlines = spark.read.csv("productlines.csv", header=True, inferSchema=True)
offices = spark.read.csv("offices.csv", header=True, inferSchema=True)
employees = spark.read.csv("employees.csv", header=True, inferSchema=True)


In [9]:
products.printSchema()
products.show(5)


root
 |-- productCode: string (nullable = true)
 |-- productName: string (nullable = true)
 |-- productLine: string (nullable = true)
 |-- productScale: string (nullable = true)
 |-- productVendor: string (nullable = true)
 |-- productDescription: string (nullable = true)
 |-- quantityInStock: string (nullable = true)
 |-- buyPrice: string (nullable = true)
 |-- MSRP: string (nullable = true)

+-----------+--------------------+------------+------------+--------------------+--------------------+---------------+--------+------+
|productCode|         productName| productLine|productScale|       productVendor|  productDescription|quantityInStock|buyPrice|  MSRP|
+-----------+--------------------+------------+------------+--------------------+--------------------+---------------+--------+------+
|   S10_1678|1969 Harley David...| Motorcycles|        1:10|     Min Lin Diecast|This replica feat...|           7933|   48.81|  95.7|
|   S10_1949|1952 Alpine Renau...|Classic Cars|        1:10|Cla

In [10]:
products.createOrReplaceTempView("products")
customers.createOrReplaceTempView("customers")
orders.createOrReplaceTempView("orders")
orderdetails.createOrReplaceTempView("orderdetails")
payments.createOrReplaceTempView("payments")


In [11]:
spark.sql("""
SELECT productLine, COUNT(*) AS total_products
FROM products
GROUP BY productLine
""").show()


+----------------+--------------+
|     productLine|total_products|
+----------------+--------------+
|     Motorcycles|            13|
|    Vintage Cars|            24|
|           Ships|             9|
|Trucks and Buses|            11|
|    Classic Cars|            38|
|          Trains|             3|
|          Planes|            12|
+----------------+--------------+



In [12]:
spark.sql("""
SELECT c.customerName, SUM(od.quantityOrdered * od.priceEach) AS total_value
FROM customers c
JOIN orders o ON c.customerNumber = o.customerNumber
JOIN orderdetails od ON o.orderNumber = od.orderNumber
GROUP BY c.customerName
ORDER BY total_value DESC
""").show()


+--------------------+------------------+
|        customerName|       total_value|
+--------------------+------------------+
|Euro+ Shopping Ch...| 820689.5399999995|
|Mini Gifts Distri...|         591827.34|
|Australian Collec...|180585.06999999998|
|  Muscle Machine Inc|177913.95000000007|
|   La Rochelle Gifts|         158573.12|
|Dragon Souveniers...|         156251.03|
|Down Under Souven...|154622.08000000002|
|   Land of Toys Inc.|         149085.15|
|      AV Stores, Co.|148410.09000000003|
|The Sharp Gifts W...|         143536.27|
|Salzburg Collecta...|137480.07000000004|
|   Kelly's Gift Shop|137460.78999999995|
|Anna's Decoration...|         137034.22|
|Souveniers And Th...|133907.12000000002|
|Corporate Gift Id...|132340.77999999997|
|Saveley & Henriot...|130305.34999999998|
|Danish Wholesale ...|129085.12000000001|
|       Rovelli Gifts|         127529.69|
|  Reims Collectables|126983.18999999997|
| L'ordine Souveniers|125505.56999999999|
+--------------------+------------

In [13]:
spark.sql("""
SELECT productName, quantityInStock
FROM products
ORDER BY quantityInStock ASC
LIMIT 5
""").show()


+--------------------+--------------------+
|         productName|     quantityInStock|
+--------------------+--------------------+
|   1957 Chevy Pickup|      Rubber wheels"|
|  1969 Corvair Monza|          hood opens|
|  1970 Dodge Coronet| hood opens and r...|
|The USS Constitut...|   sea sprite on bow|
|1970 Chevy Chevel...|                1005|
+--------------------+--------------------+



In [14]:
spark.sql("""
SELECT customerNumber, SUM(amount) AS total_payment
FROM payments
GROUP BY customerNumber
ORDER BY total_payment DESC
LIMIT 5
""").show()


+--------------+------------------+
|customerNumber|     total_payment|
+--------------+------------------+
|           141| 715738.9800000001|
|           124| 584188.2400000001|
|           114|180585.06999999998|
|           151|         177913.95|
|           148|         156251.03|
+--------------+------------------+



In [15]:
spark.sql("""
SELECT DATE_FORMAT(orderDate, 'yyyy-MM') AS month, COUNT(*) AS total_orders
FROM orders
GROUP BY month
ORDER BY month
""").show()


+-------+------------+
|  month|total_orders|
+-------+------------+
|2003-01|           5|
|2003-02|           3|
|2003-03|           6|
|2003-04|           7|
|2003-05|           6|
|2003-06|           7|
|2003-07|           7|
|2003-08|           5|
|2003-09|           8|
|2003-10|          18|
|2003-11|          30|
|2003-12|           9|
|2004-01|           8|
|2004-02|          11|
|2004-03|           8|
|2004-04|          10|
|2004-05|           8|
|2004-06|          12|
|2004-07|          11|
|2004-08|          12|
+-------+------------+
only showing top 20 rows



In [16]:
result_df = spark.sql("""
SELECT productLine, COUNT(*) AS total_products
FROM products
GROUP BY productLine
""")
result_df.write.csv("output/product_count", header=True)


In [17]:
!zip -r output.zip output
from google.colab import files
files.download("output.zip")


  adding: output/ (stored 0%)
  adding: output/product_count/ (stored 0%)
  adding: output/product_count/._SUCCESS.crc (stored 0%)
  adding: output/product_count/_SUCCESS (stored 0%)
  adding: output/product_count/.part-00000-8953c16b-13d8-4996-8b9d-e1abf8cfb6c7-c000.csv.crc (stored 0%)
  adding: output/product_count/part-00000-8953c16b-13d8-4996-8b9d-e1abf8cfb6c7-c000.csv (deflated 12%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [18]:
spark.sql("""
SELECT p.productName, SUM(od.quantityOrdered) AS total_qty,
       SUM(od.quantityOrdered*od.priceEach) AS total_value
FROM orderdetails od
JOIN products p ON od.productCode = p.productCode
GROUP BY p.productName
ORDER BY total_qty DESC
LIMIT 1
""").show()


+--------------------+---------+-----------+
|         productName|total_qty|total_value|
+--------------------+---------+-----------+
|1992 Ferrari 360 ...|     1808|  276839.98|
+--------------------+---------+-----------+



In [19]:
payments.createOrReplaceTempView("payments")
customers.createOrReplaceTempView("customers")
employees.createOrReplaceTempView("employees")
offices.createOrReplaceTempView("offices")
orders.createOrReplaceTempView("orders")


In [20]:
spark.sql("""
SELECT o.officeCode,
       ROUND(AVG(DATEDIFF(p.paymentDate, o2.shippedDate)), 2) AS avg_delay_days
FROM payments p
JOIN customers c ON p.customerNumber = c.customerNumber
JOIN employees e ON c.salesRepEmployeeNumber = e.employeeNumber
JOIN offices o ON e.officeCode = o.officeCode
JOIN orders o2 ON c.customerNumber = o2.customerNumber
WHERE p.paymentDate IS NOT NULL AND o2.shippedDate IS NOT NULL
GROUP BY o.officeCode
ORDER BY avg_delay_days DESC
""").show()


+----------+--------------+
|officeCode|avg_delay_days|
+----------+--------------+
|         5|         21.61|
|         2|         12.76|
|         3|         10.17|
|         7|           4.8|
|         4|        -14.29|
|         6|        -25.42|
|         1|        -28.41|
+----------+--------------+

